In [ ]:
import cv2
from ultralytics import YOLO
import traceback
import numpy as np

# First Attempt at Running Model From Camera

In [ ]:
model = YOLO('yolov8m-pose.pt')
results = model(source=0, show=True, conf=.3, save=False)

This code has a few problems. First there is no way to analyze individual frames and secondly, the video can't be stopped until you reset the kernel.

# Keypoints and Angles Between Joints

I want to get each joint from each individual frame and calculate the angles for each joint. Before I had no way of getting data on a single frame, however, if I use openCV to capture from my camera I can pass in the frames individually. After running the YOLOv8 model on a single frame it returns its results. From this you are able to get the 17 keypoints making up the pose. Each of these points represent the pixel coordinates of each join. The points are all stored in an array in the following order:

0. nose
1. left eye 
2. right eye
3. left ear
4. right ear
5. left shoulder
6. right shoulder
7. left elbow
8. right elbow
9. left wrist
10. right wrist
11. left waist
12. right waist
13. left knee
14. right knee
15. left ankle
16. right ankle

## Calculating Angles

In [17]:
def calc_angle(point_A, point_B, point_C):
    # Calculate vectors from B to A and B to C
    vector_AB = point_A - point_B
    vector_BC = point_C - point_B

    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector_AB, vector_BC)

    # Calculate the magnitudes (lengths) of the vectors
    magnitude_AB = np.linalg.norm(vector_AB)
    magnitude_BC = np.linalg.norm(vector_BC)

    # Calculate the cosine of the angle θ using the dot product formula
    cosine_theta = dot_product / (magnitude_AB * magnitude_BC)

    # Calculate the angle θ in radians using the arccosine function
    angle_radians = np.arccos(cosine_theta)

    # Convert the angle from radians to degrees
    angle_degrees = np.degrees(angle_radians)

    return (angle_degrees, angle_radians)

## Annotating Angles on Capture

In [18]:
# Angle indices
joint_angle_indices = {
    "left_elbow": [5, 7, 9],
    "right_elbow": [6, 8, 10],
    "left_shoulder": [11, 5, 7],
    "right_shoulder": [12, 6, 8],
    "left_waist": [5, 11, 13],
    "right_waist": [6, 12, 14],
    "left_knee": [11, 13, 15],
    "right_knee": [12, 14, 16]
}

# Load the YOLOv8 model
model = YOLO('yolov8m-pose.pt')

# Start video capture
cap = cv2.VideoCapture(0)

try:
    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            results = model(source=frame, conf=.3)

            # Visualize the pose results on the frame
            annotated_frame = results[0].plot()

            # Get keypoints from results
            pixel_keypoints = results[0].keypoints.xy[0]

            # Calculate and annotate the angle of each joints
            if (len(pixel_keypoints) != 0):
                for joint_name in joint_angle_indices:
                    indices = joint_angle_indices[joint_name]
                    img_points = [np.array(pixel_keypoints[i]) for i in indices]
                    cv2.putText(
                        annotated_frame,
                        str(calc_angle(*img_points)[0]),
                        tuple(map(int, img_points[1])),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 1
                    )

            # Display the annotated frame
            cv2.imshow("YOLOv8 Inference", annotated_frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
except Exception as err:
    traceback.print_exc()
finally:
    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()




0: 480x640 1 person, 335.6ms
Speed: 2.0ms preprocess, 335.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 347.9ms
Speed: 1.1ms preprocess, 347.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 318.2ms
Speed: 1.0ms preprocess, 318.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 325.6ms
Speed: 1.0ms preprocess, 325.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 310.2ms
Speed: 1.0ms preprocess, 310.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 319.6ms
Speed: 1.0ms preprocess, 319.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 315.1ms
Speed: 1.0ms preprocess, 315.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 319.6ms
Speed: 1.0ms preprocess, 319.6ms inference, 1.0ms postprocess per image at